In [1]:
import tensorflow as tf

In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [7]:
layer = tf.keras.layers.Dense(100)

layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [8]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [10]:
layer.variables

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.1108216 , -0.16462934, -0.4856644 , -0.10724229,  0.08261704,
         -0.61132365,  0.42282766, -0.34958082, -0.58981687,  0.2728204 ],
        [-0.03531814,  0.6176669 ,  0.05374098,  0.5286619 , -0.08286393,
          0.06548619,  0.00367713, -0.10622191, -0.6121604 ,  0.14151937],
        [ 0.5645923 ,  0.21425748,  0.02452147,  0.22111386, -0.6279765 ,
         -0.04564536, -0.47447556, -0.48075968, -0.60557705,  0.24708039],
        [ 0.18437201, -0.4269103 , -0.27437803, -0.01332551, -0.5685719 ,
          0.26905566,  0.35586345, -0.24684426,  0.5982682 ,  0.50852484],
        [ 0.27326012, -0.29130173, -0.03060794,  0.16582602, -0.21027547,
          0.48613256,  0.60180765, -0.30091467, -0.51551324, -0.43273002]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [11]:
layer.kernel, layer.bias

(<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.1108216 , -0.16462934, -0.4856644 , -0.10724229,  0.08261704,
         -0.61132365,  0.42282766, -0.34958082, -0.58981687,  0.2728204 ],
        [-0.03531814,  0.6176669 ,  0.05374098,  0.5286619 , -0.08286393,
          0.06548619,  0.00367713, -0.10622191, -0.6121604 ,  0.14151937],
        [ 0.5645923 ,  0.21425748,  0.02452147,  0.22111386, -0.6279765 ,
         -0.04564536, -0.47447556, -0.48075968, -0.60557705,  0.24708039],
        [ 0.18437201, -0.4269103 , -0.27437803, -0.01332551, -0.5685719 ,
          0.26905566,  0.35586345, -0.24684426,  0.5982682 ,  0.50852484],
        [ 0.27326012, -0.29130173, -0.03060794,  0.16582602, -0.21027547,
          0.48613256,  0.60180765, -0.30091467, -0.51551324, -0.43273002]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [12]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
  
  def build(self, input_shape):
    self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])
  
  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [13]:
_ = layer(tf.zeros([10, 5]))

In [15]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [16]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(input_tensor)
    x = self.bn2c(x, training=training)
    
    x += input_tensor
    return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])

In [17]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [18]:
block.layers

In [19]:
len(block.variables)

18

In [20]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  12        
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 44
Trainable params: 32
Non-trainable params: 12
________________________________________________

In [21]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2, 1, padding='same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1, 1)),
                              tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [22]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________